In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Design Workflow for Two-Photon Lithography\n",
    "\n",
    "This notebook demonstrates the complete design workflow from geometry creation to toolpath generation.\n",
    "\n",
    "**Author**: Zeyad Mustafa  \n",
    "**Institution**: BTU Cottbus-Senftenberg  \n",
    "**Date**: December 2024"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Setup\n",
    "\n",
    "Import required modules and configure notebook."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Standard imports\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from pathlib import Path\n",
    "\n",
    "# TPL imports\n",
    "from tpl.design import Geometry, Cube, Sphere, Cylinder, PathPlanner\n",
    "\n",
    "# Configure matplotlib\n",
    "%matplotlib inline\n",
    "plt.rcParams['figure.figsize'] = (12, 8)\n",
    "\n",
    "print(\"✓ Imports successful\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Creating Basic Geometries\n",
    "\n",
    "Let's create some basic 3D shapes."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a cube\n",
    "cube = Cube(size=10, center=(0, 0, 10))\n",
    "\n",
    "print(f\"Cube created:\")\n",
    "print(f\"  Volume: {cube.get_volume():.2f} μm³\")\n",
    "print(f\"  Bounds: {cube.get_bounds()}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a sphere\n",
    "sphere = Sphere(radius=5, center=(15, 0, 10))\n",
    "\n",
    "print(f\"Sphere created:\")\n",
    "print(f\"  Volume: {sphere.get_volume():.2f} μm³\")\n",
    "print(f\"  Radius: {sphere.radius} μm\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a cylinder\n",
    "cylinder = Cylinder(radius=3, height=20, center=(30, 0, 10))\n",
    "\n",
    "print(f\"Cylinder created:\")\n",
    "print(f\"  Volume: {cylinder.get_volume():.2f} μm³\")\n",
    "print(f\"  Height: {cylinder.height} μm\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Boolean Operations\n",
    "\n",
    "Combine shapes using boolean operations."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create two overlapping cubes\n",
    "cube1 = Cube(size=10, center=(0, 0, 10))\n",
    "cube2 = Cube(size=10, center=(5, 0, 10))\n",
    "\n",
    "# Union\n",
    "union = cube1.union(cube2)\n",
    "print(f\"Union volume: {union.get_volume():.2f} μm³\")\n",
    "print(f\"  (Original: {cube1.get_volume()*2:.2f} μm³)\")\n",
    "\n",
    "# Intersection\n",
    "intersection = cube1.intersection(cube2)\n",
    "print(f\"Intersection volume: {intersection.get_volume():.2f} μm³\")\n",
    "\n",
    "# Difference (hollow cube)\n",
    "outer = Cube(size=10, center=(0, 0, 10))\n",
    "inner = Cube(size=8, center=(0, 0, 10))\n",
    "hollow = outer.difference(inner)\n",
    "print(f\"Hollow cube volume: {hollow.get_volume():.2f} μm³\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Transformations\n",
    "\n",
    "Apply transformations to compensate for shrinkage."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create cube\n",
    "cube = Cube(size=10, center=(0, 0, 10))\n",
    "print(f\"Original volume: {cube.get_volume():.2f} μm³\")\n",
    "\n",
    "# Apply shrinkage compensation (IP-Dip: ~12% volumetric)\n",
    "# Lateral: 5%, Axial: 11%\n",
    "cube.scale(1.05, 1.05, 1.11)\n",
    "print(f\"After compensation: {cube.get_volume():.2f} μm³\")\n",
    "\n",
    "# Translation\n",
    "translation = np.eye(4)\n",
    "translation[:3, 3] = [5, 5, 0]\n",
    "cube.transform(translation)\n",
    "print(f\"After translation: {cube.get_bounds()}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Path Planning\n",
    "\n",
    "Generate fabrication toolpath from geometry."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create test geometry\n",
    "test_cube = Cube(size=10, center=(0, 0, 10))\n",
    "\n",
    "# Create path planner\n",
    "planner = PathPlanner(\n",
    "    layer_height=0.5,\n",
    "    hatch_distance=0.5,\n",
    "    scan_speed=50000,\n",
    "    power=20,\n",
    "    fill_pattern=\"rectilinear\"\n",
    ")\n",
    "\n",
    "# Set first layer parameters\n",
    "planner.set_first_layer(power=26, speed=25000)\n",
    "\n",
    "print(\"Path planner configured\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Generate toolpath\n",
    "toolpath = planner.generate(test_cube)\n",
    "\n",
    "# Get statistics\n",
    "stats = toolpath.get_statistics()\n",
    "\n",
    "print(\"Toolpath Statistics:\")\n",
    "print(f\"  Total points: {stats['num_points']}\")\n",
    "print(f\"  Number of layers: {stats['num_layers']}\")\n",
    "print(f\"  Total length: {stats['total_length']:.1f} μm\")\n",
    "print(f\"  Estimated time: {stats['time_estimate']:.1f} s ({stats['time_estimate']/60:.1f} min)\")\n",
    "print(f\"  Power range: {stats['min_power']:.1f} - {stats['max_power']:.1f} mW\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Visualization\n",
    "\n",
    "Visualize the toolpath."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Get coordinates\n",
    "coords = toolpath.get_coordinates()\n",
    "\n",
    "# Create 2D projections\n",
    "fig, axes = plt.subplots(1, 3, figsize=(15, 5))\n",
    "\n",
    "# XY projection (top view)\n",
    "axes[0].scatter(coords[:, 0], coords[:, 1], c=coords[:, 2], cmap='viridis', s=1)\n",
    "axes[0].set_xlabel('X (μm)')\n",
    "axes[0].set_ylabel('Y (μm)')\n",
    "axes[0].set_title('Top View (XY)')\n",
    "axes[0].set_aspect('equal')\n",
    "axes[0].grid(True, alpha=0.3)\n",
    "\n",
    "# XZ projection (side view)\n",
    "axes[1].scatter(coords[:, 0], coords[:, 2], c=coords[:, 1], cmap='viridis', s=1)\n",
    "axes[1].set_xlabel('X (μm)')\n",
    "axes[1].set_ylabel('Z (μm)')\n",
    "axes[1].set_title('Side View (XZ)')\n",
    "axes[1].set_aspect('equal')\n",
    "axes[1].grid(True, alpha=0.3)\n",
    "\n",
    "# YZ projection (front view)\n",
    "axes[2].scatter(coords[:, 1], coords[:, 2], c=coords[:, 0], cmap='viridis', s=1)\n",
    "axes[2].set_xlabel('Y (μm)')\n",
    "axes[2].set_ylabel('Z (μm)')\n",
    "axes[2].set_title('Front View (YZ)')\n",
    "axes[2].set_aspect('equal')\n",
    "axes[2].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Export Files\n",
    "\n",
    "Save geometry and toolpath for fabrication."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create output directory\n",
    "output_dir = Path(\"notebook_output\")\n",
    "output_dir.mkdir(exist_ok=True)\n",
    "\n",
    "# Save geometry\n",
    "test_cube.save(output_dir / \"cube.stl\")\n",
    "print(f\"✓ Geometry saved: {output_dir}/cube.stl\")\n",
    "\n",
    "# Save toolpath\n",
    "toolpath.save(output_dir / \"toolpath.gcode\")\n",
    "print(f\"✓ Toolpath saved: {output_dir}/toolpath.gcode\")\n",
    "\n",
    "# Save coordinates\n",
    "toolpath.export_to_csv(output_dir / \"coordinates.csv\")\n",
    "print(f\"✓ Coordinates saved: {output_dir}/coordinates.csv\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Comparing Fill Patterns\n",
    "\n",
    "Test different fill patterns and compare."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Test different fill patterns\n",
    "patterns = ['rectilinear', 'concentric', 'spiral']\n",
    "results = {}\n",
    "\n",
    "for pattern in patterns:\n",
    "    planner = PathPlanner(\n",
    "        layer_height=0.5,\n",
    "        hatch_distance=0.5,\n",
    "        fill_pattern=pattern\n",
    "    )\n",
    "    \n",
    "    toolpath = planner.generate(test_cube)\n",
    "    stats = toolpath.get_statistics()\n",
    "    \n",
    "    results[pattern] = {\n",
    "        'points': stats['num_points'],\n",
    "        'length': stats['total_length'],\n",
    "        'time': stats['time_estimate']\n",
    "    }\n",
    "\n",
    "# Display comparison\n",
    "import pandas as pd\n",
    "df = pd.DataFrame(results).T\n",
    "print(\"\\nFill Pattern Comparison:\")\n",
    "print(df)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Summary\n",
    "\n",
    "This notebook demonstrated:\n",
    "\n",
    "1. ✓ Creating basic geometries (Cube, Sphere, Cylinder)\n",
    "2. ✓ Boolean operations (union, intersection, difference)\n",
    "3. ✓ Transformations (scaling, translation)\n",
    "4. ✓ Path planning with different strategies\n",
    "5. ✓ Toolpath visualization\n",
    "6. ✓ File export (STL, G-code, CSV)\n",
    "7. ✓ Comparing fill patterns\n",
    "\n",
    "**Next Steps**:\n",
    "- Try `02_parameter_optimization.ipynb` for parameter tuning\n",
    "- Explore more complex geometries\n",
    "- Test with real hardware"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}